In [1]:
import re
import os
import pandas as pd

file_paths, file_names, emotions, audios = [], [], [], []
emotion_map = {'anger': 'angry', 'happiness': 'happy', 'sadness': 'sad', 'fear': 'fear',
              'disgust': 'disgust'}

In [2]:
import torch
import torchaudio

print(torch.__version__)
print(torchaudio.__version__)

torch.random.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

1.12.1+cu113
0.12.1+cu113
cuda


In [3]:
bundle = torchaudio.pipelines.WAV2VEC2_LARGE
extractor = bundle.get_model()
print(extractor.__class__)
print(bundle.sample_rate)

<class 'torchaudio.models.wav2vec2.model.Wav2Vec2Model'>
16000


In [4]:
# import os
# os.listdir('../emotiondata/emotion_data')

In [5]:
# folder_list = ['anger', 'disgust', 'fear', 'happiness', 'sadness']
# entries = []
# for folder in folder_list:
#     cur_file_list = os.listdir(f'../emotiondata/emotion_data/{folder}')
#     for i in cur_file_list:
#         if i == 's05 (3).wav':
#             print("found")
#             continue
#         entries.append(i)

In [6]:
# import random
# random.shuffle(entries)
# session = []
# equal_parts = (len(entries)-1)//5 # for equally split the entries into 5 parts
# count = 0
# main_data_path = '../emotiondata/emotion_data'

In [7]:
# ### Only Run once
# from tqdm import tqdm 
# folder_map = {'a':'anger', 'd':'disgust', 'f':'fear', 'h':'happiness', 's':'sadness'}

# file_paths = []
# file_names = []
# emotions = []
# # audios = []
# # labels = []


# for i in tqdm(range(len(entries))):
#     entry = entries[i]
#     if "wav" not in entry:
#         continue
#     folder = folder_map[entry[0]]
#     file_path = f'../emotiondata/emotion_data/{folder}/{entry}'
#     emotion = emotion_map[folder]
#     file_paths.append(file_path)
#     file_names.append(entry)
#     emotions.append(emotion)

#     # assign session to it
#     part = (count//equal_parts)%6 + 1
#     if part == 6:
#         part = 5
#     session.append(part)
#     count += 1


# file = pd.DataFrame({'path':file_paths, 'name': file_names, 'emotion': emotions, 'session': session})
# dataframe_path = main_data_path + '/session_entries.csv'
# file.to_csv(dataframe_path)


In [8]:
# os.listdir('../emotiondata/emotion_data')

# Extract Features using Models

In [9]:
import re
import os
import pandas as pd
main_data_path = '../emotiondata/emotion_data'
dataframe_path = main_data_path + '/session_entries.csv'
file = pd.read_csv(dataframe_path)[['path', 'name', 'emotion', 'session']]
file.head()

,path,name,emotion,session
0,../emotiondata/emotion_data/happiness/h17 (5).wav,h17 (5).wav,happy,1
1,../emotiondata/emotion_data/happiness/h20 (2).wav,h20 (2).wav,happy,1
2,../emotiondata/emotion_data/disgust/d19 (4).wav,d19 (4).wav,disgust,1
3,../emotiondata/emotion_data/sadness/s12 (4).wav,s12 (4).wav,sad,1
4,../emotiondata/emotion_data/sadness/s01 (4).wav,s01 (4).wav,sad,1


In [10]:
from tqdm import tqdm
import math
audios = []
ct = 0
for i in tqdm(range(math.floor(len(file['path'])))):
    path = file['path'][i]
    wave, sr = torchaudio.load(path)
    if sr != bundle.sample_rate:
        wave = torchaudio.functional.resample(wave, sr, bundle.sample_rate)
    with torch.inference_mode():
        feature, _ = extractor.extract_features(wave)
    
    feature = [f[0] for f in feature]
    audio = torch.stack(feature)
    audios.append(audio)
    ct += 1

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 604/604 [08:24<00:00,  1.20it/s]


# Load Data

In [49]:
holdout = 5
train = file[file['session'] != holdout]
train_audios = [audios[i] for i in range(len(audios)) if file['session'][i] != holdout]
test = file[file['session'] == holdout]
test_audios = [audios[i] for i in range(len(audios)) if file['session'][i] == holdout]

In [50]:
class MyDataSet(torch.utils.data.Dataset):
    def __init__(self, audios, labels, label_transform):
        super(MyDataSet).__init__()
        self.audios = audios
        self.labels = labels
        self.label_transform = label_transform
        
    def __getitem__(self, idx):
        label = self.label_transform[self.labels[idx]]
        audio = self.audios[idx]
        length = audio.size(1)
        return audio, length, label
    
    def __len__(self):
        return len(self.labels)

In [51]:
def collate_indic(data):
    audios, lengths, labels = zip(*data)
    max_len = max(lengths)
    n_ftrs = audios[0].size(2)
    n_dims = audios[0].size(0)
    features = torch.zeros((len(audios), n_dims, max_len, n_ftrs))
    labels = torch.tensor(labels)
    lengths = torch.tensor(lengths)

    for i in range(len(data)):
        j, k = audios[i].size(1), audios[i].size(2)
        features[i] = torch.cat([audios[i], torch.zeros((n_dims, max_len - j, k))], dim=1)

    return features, lengths, labels

In [52]:
categories = ['angry', 'happy', 'sad', 'fear', 'disgust']
cate_dic = {}
for i, cate in enumerate(categories):
    cate_dic[cate] = i
cate_dic

{'angry': 0, 'happy': 1, 'sad': 2, 'fear': 3, 'disgust': 4}

In [53]:
from torch.utils.data import DataLoader

train_dataset = MyDataSet(train_audios, train['emotion'].tolist(), cate_dic)
trainloader_args = dict(batch_size=16, shuffle=True)
train_dataloader = DataLoader(train_dataset, **trainloader_args, 
                              collate_fn=collate_indic)

test_dataset = MyDataSet(test_audios, test['emotion'].tolist(), cate_dic)
testloader_args = dict(batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, **testloader_args, 
                             collate_fn=collate_indic)

# 3CNN+LSTM

In [54]:
import torch.nn as nn
import torch.nn.functional as F

class ICASSP3CNN(nn.Module):
    def __init__(self, vocab_size, dims = 12, embed_size=128, hidden_size=512, num_lstm_layers = 2, 
                 bidirectional = False, label_size = 5):
        super().__init__()
        self.n_layers = num_lstm_layers 
        self.hidden = hidden_size
        self.bidirectional = bidirectional
        
        self.aggr = nn.Conv1d(in_channels=dims, out_channels=1, kernel_size=1)
        
        self.embed = nn.Linear(in_features = vocab_size, out_features = embed_size)

        self.cnn  = nn.Conv1d(embed_size, embed_size, kernel_size=3, padding=1)
        self.cnn2 = nn.Conv1d(embed_size, embed_size, kernel_size=5, padding=2)
        self.cnn3 = nn.Conv1d(embed_size, embed_size, kernel_size=7, padding=3)

        self.batchnorm = nn.BatchNorm1d(3 * embed_size)

        self.lstm = nn.LSTM(input_size = 3 * embed_size, 
                            hidden_size = hidden_size, 
                            num_layers = num_lstm_layers, 
                            bidirectional = bidirectional)

        self.linear = nn.Linear(in_features = 2 * hidden_size if bidirectional else hidden_size, 
                                out_features = label_size)


    def forward(self, x, lengths):
        """
        padded_x: (B,T) padded LongTensor
        """
        n, d, b, t = x.size(0), x.size(1), x.size(2), x.size(3)
        x = torch.flatten(x, start_dim=2)
        input = self.aggr(x)
        input = torch.reshape(input, (n, b, t))
        input = self.embed(input)

        batch_size = input.size(0)
        input = input.transpose(1,2)    # (B,T,H) -> (B,H,T)

        cnn_output = torch.cat([self.cnn(input), self.cnn2(input), self.cnn3(input)], dim=1)

        input = F.relu(self.batchnorm(cnn_output))

        input = input.transpose(1,2)

        pack_tensor = nn.utils.rnn.pack_padded_sequence(input, lengths, batch_first=True, enforce_sorted=False)
        _, (hn, cn) = self.lstm(pack_tensor)

        if self.bidirectional:
            h_n = hn.view(self.n_layers, 2, batch_size, self.hidden)
            h_n = torch.cat([ h_n[-1, 0,:], h_n[-1,1,:] ], dim = 1)
        else:
            h_n = hn[-1]

        logits = self.linear(h_n)

        return logits


In [55]:
from tqdm import tqdm
from torchsummary import summary
import torch.optim as optim

model = ICASSP3CNN(1024, 24)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


epochs = 50
train_losses = []
train_accuracies = []
valid_losses = []
valid_accuracies = []

for epoch in tqdm(range(epochs)):
    train_loss = 0
    acc_cnt = 0
    err_cnt = 0
    batch_cnt = 0
    model.train()
    for batch, (x, length, y) in enumerate(train_dataloader):
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        logits = model(x, length)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.cpu().item()

        #model outputs
        out_val, out_indices = torch.max(logits, dim=1)
        tar_indices = y

        for i in range(len(out_indices)):
            if out_indices[i] == tar_indices[i]:
                acc_cnt += 1
            else:
                err_cnt += 1
        batch_cnt += 1
    
    train_loss = train_loss/batch_cnt
    train_accuracy = acc_cnt/(acc_cnt+err_cnt)
    train_accuracies.append(train_accuracy)
    train_losses.append(train_loss)
    
    print(f"epoch:{epoch+1}, train accu:{train_accuracy:.4f},", f"train loss:{train_loss:.2f}")

  2%|█████▋                                                                                                                                                                                                                                                                                         | 1/50 [00:11<09:11, 11.26s/it]

epoch:1, train accu:0.4604, train loss:1.25


  4%|███████████▍                                                                                                                                                                                                                                                                                   | 2/50 [00:21<08:42, 10.88s/it]

epoch:2, train accu:0.6896, train loss:0.76


  6%|█████████████████▏                                                                                                                                                                                                                                                                             | 3/50 [00:31<08:14, 10.52s/it]

...

epoch:48, train accu:1.0000, train loss:0.00


 98%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎     | 49/50 [10:20<00:10, 10.48s/it]

epoch:49, train accu:1.0000, train loss:0.00


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [10:30<00:00, 12.62s/it]

epoch:50, train accu:1.0000, train loss:0.00


# Model Test

In [56]:
test_loss = 0
acc_cnt = 0
err_cnt = 0
batch_cnt = 0
model.eval()

for x, lengths, y in test_dataloader:

    x = x.to(device)
    y = y.to(device)

    logits = model(x, lengths)
    loss = criterion(logits, y)
    test_loss += loss.cpu().item()

    out_val, out_indices = torch.max(logits, dim=1)
    tar_indices = y

    for i in range(len(out_indices)):
        if out_indices[i] == tar_indices[i]:
            acc_cnt += 1
        else:
            err_cnt += 1
    batch_cnt += 1

test_loss = test_loss/batch_cnt
test_accuracy = acc_cnt/(acc_cnt+err_cnt)
print(f'test accuracy: {test_accuracy}')

test accuracy: 0.8870967741935484


In [57]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)

aggr.weight tensor([[[ 0.1121],
         [-0.0618],
         [-0.0135],
         [-0.1137],
         [ 0.2014],
         [ 0.0477],
         [-0.1968],
         [-0.0321],
         [-0.2502],
         [ 0.0935],
         [ 0.1387],
         [ 0.1686],
         [-0.0087],
         [-0.0973],
         [-0.0934],
         [ 0.1011],
         [-0.0434],
         [ 0.1706],
         [ 0.0207],
         [ 0.1378],
         [-0.2227],
         [ 0.1070],
         [-0.1443],
         [ 0.0210]]], device='cuda:0')
aggr.bias tensor([0.0927], device='cuda:0')
embed.weight tensor([[-0.0019, -0.0293, -0.0095,  ..., -0.0395, -0.0133,  0.0065],
        [-0.0036, -0.0075,  0.0286,  ..., -0.0105,  0.0036,  0.0491],
        [-0.0498, -0.0146, -0.0213,  ...,  0.0379,  0.0543,  0.0445],
        ...,
        [ 0.0093,  0.0468, -0.0403,  ...,  0.0255,  0.0395, -0.0153],
        [-0.0047,  0.0399, -0.0018,  ...,  0.0185, -0.0548, -0.0220],
        [ 0.0728, -0.0321, -0.0253,  ..., -0.0189, -0.0214, -0.0564]]

In [ ]:
# model_path = main_path + f'/models/wav2veclarge/holdout_{holdout}.pth'

# torch.save({'epoch':epochs,
#             'model_state_dict':model.state_dict(),
#             'optimizer_state_dict':optimizer.state_dict()},
#             model_path)